In [ ]:
import csv
import sys
import requests
import skimage.io
import os
import glob
import pickle
import time

from IPython.display import display, Image, HTML
from keras.applications import VGG16
import datetime
from shutil import copyfile


In [ ]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image as kimage
import numpy as np
import pandas as pd
import scipy.sparse as sp

import skimage.io
import hashlib

#sys.path.append('../')
#import helpers

In [ ]:
#não rodar se já tiver o arqs.pickle

def calcula_md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def get_arquivos_hashs(fonte, arqs):
    raiz = arqs[fonte]["raiz"]
    extensaoimg=arqs["extensoes_img"]
    print("\nCalcula hashs dos arquivos", fonte)
    print("Diretório raiz", raiz)
    num = 0
    for r,d,f in os.walk(raiz):
        for arquivo_rel in f:
            arquivo_abs = os.path.join(r, arquivo_rel)
            arquivo_rel = arquivo_abs[len(raiz):]
            ind = arquivo_rel.rfind(".")
            #num+=1
            #if (num%500==0): 
            #    z=input("digite algo: ")
            #    if z=="q": return
            if ind>0 and os.path.isfile(arquivo_abs):
                #print(arquivo_abs, ext)
                ext = arquivo_rel[ind:].lower()
                if ext in extensaoimg:
                    md5 = calcula_md5(arquivo_abs)
                    try:
                        img = skimage.io.imread(arquivo_abs)
                        arqs[fonte]["nomes"].append(arquivo_rel)
                        if md5 not in arqs[fonte]["hashs"]:
                            arqs[fonte]["hashs"][md5] = []
                        arqs[fonte]["hashs"][md5].append(arquivo_rel)
                        num+=1
                        if (num%500==0): print("  -->",num)
                    except:
                        print("Erro arquivo de imagem:", arquivo_abs)
    print("  -->",num)


arqs = {}
arqs["busca"] = {}
arqs["material"] = {}
arqs["extensoes_img"] = ['.jpg', '.png', '.tif', '.jpeg', '.gif', '.bmp']

arqs["busca"]["raiz"] = "./img_busca/"
arqs["busca"]["nome_exibicao"] = "CD Anexo"
arqs["busca"]["nomes"] = []
arqs["busca"]["hashs"] = {}

arqs["material"]["raiz"] = "./img_quest/"
arqs["material"]["nome_exibicao"] = "HD Examinado"
arqs["material"]["nomes"] = []
arqs["material"]["hashs"] = {}


get_arquivos_hashs("busca", arqs)
get_arquivos_hashs("material", arqs)

print("busca keys = ", len(arqs["busca"]["hashs"].keys() ) )
print("busca nomes = ",len(arqs["busca"]["nomes"]))
print("material keys = ", len(arqs["material"]["hashs"].keys() ) )
print("material nomes = ",len(arqs["material"]["nomes"]))


def grava_arqs():
    filehandler = open("arqs.pickle", 'wb') 
    pickle.dump(arqs, filehandler)

grava_arqs()


In [ ]:
def grava_arqs():
    filehandler = open("arqs.pickle", 'wb') 
    pickle.dump(arqs, filehandler)

grava_arqs()


In [ ]:
# estrutura de arqs

for s in arqs.keys():
    print("\narqs[\"" + s + "\"]")
    if 'dict' in str(type(arqs[s])):
        for t in arqs[s].keys():
            print("  arqs[\"" + s + "\"][\"" + t + "\"]  ---  tipo:", type(arqs[s][t]))
        


In [ ]:
arqs["busca"]["nomes"][0]

In [ ]:

def le_arqs():
    filehandler=open("arqs.pickle", 'rb') 
    ret = pickle.load(filehandler)
    return ret

arqs = le_arqs()


In [ ]:
import os.path

rand_img = os.path.join(arqs["busca"]["raiz"] ,np.random.choice(arqs["busca"]["nomes"]))
#rand_img = np.random.choice(glob.glob('/media/pcf13/D_4TB/imagens/Mat2675_2017/*.jpg'))
print(rand_img)

print(os.path.isfile(rand_img))


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(rand_img)
imgplot = plt.imshow(img)
plt.show()


In [ ]:
img.shape[0:2]

In [ ]:
try:
    display(Image(filename=rand_img, embed=True))
except:
    print("erro de leitura do arquivo", rand_img)

In [ ]:
try:
    img = skimage.io.imread(rand_img)
    print(img.shape)
except:
    print("erro de leitura do arquivo", rand_img)    

In [ ]:
img = kimage.load_img(rand_img, target_size=(224, 224))
x = kimage.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print(x.shape)

In [ ]:
# image_top=False removes final connected layers
model = VGG16(include_top=False, weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [ ]:
pred = model.predict(x)
print(pred.shape)
print(pred.ravel().shape)

In [ ]:
%%timeit -n5
pred = model.predict(x)

In [ ]:
dados = {}
dados["busca"]    = {}
dados["material"] = {}
dados["total"]    = {}

dados["busca"]["itens"] = []
dados["busca"]["nomes"] = []

dados["material"]["itens"] = []
dados["material"]["nomes"] = []

dados["total"]["itens"] = []
dados["total"]["nomes"] = []

dados["predicoes"]    = None
dados["predicoescsr"] = None
dados["sim"]          = None

#for s in dados.keys():
#    print("\ndados[\"" + s + "\"]")
#    if 'dict' in str(type(dados[s])):
#        for t in dados[s].keys():
#            print("  dados[\"" + s + "\"][\"" + t + "\"]  ---  tipo:", type(dados[s][t]))
        

In [ ]:
dados["busca"]["itens"]=list(arqs["busca"]["hashs"].items())
dados["busca"]["nomes"] = [os.path.join(arqs["busca"]["raiz"] ,f[1][0]) for f in dados["busca"]["itens"]]

dados["material"]["itens"]=list(arqs["material"]["hashs"].items())
dados["material"]["nomes"] = [os.path.join(arqs["material"]["raiz"] ,f[1][0]) for f in dados["material"]["itens"]]

dados["total"]["itens"] = dados["busca"]["itens"] + dados["material"]["itens"]
dados["total"]["nomes"] = dados["busca"]["nomes"] + dados["material"]["nomes"]

In [ ]:
#sanity check 

for i in range(0, len(dados["busca"]["itens"])):
    x1 = dados["busca"]["itens"][i]
    x2 = dados["busca"]["nomes"][i]
    y1 = dados["total"]["itens"][i]
    y2 = dados["total"]["nomes"][i]
    if (x1[0]!=y1[0]): print(x1[0], y1[0]) #nao deve imprimir nunca
    if (x1[1]!=y1[1]): print(x1[1], y1[1]) #nao deve imprimir nunca
    if (x2!=y2): print(x1, y2)             #nao deve imprimir nunca

#limites
print(dados["total"]["nomes"][i])
print(dados["total"]["nomes"][i+1])
print(dados["total"]["itens"][i])
print(dados["total"]["itens"][i+1])

In [ ]:
#NAO EXECUTAR SE PUDER CARREGAR DO PICKLE

batch_size = 500
min_idx = 0
total_max = len(dados["total"]["nomes"])
max_idx = min(min_idx + batch_size, total_max)
n_dims = pred.ravel().shape[0]
px = 224

# Initialize predictions matrix
dados["predicoes"] = sp.lil_matrix((len(dados["total"]["nomes"]), n_dims))

while min_idx < total_max - 1:
    
    X = np.zeros(((max_idx - min_idx), px, px, 3))
    
    # carrega arquivos em X
    for i in range(min_idx, max_idx):
        fname = dados["total"]["nomes"][i]
        # carrega imagem e converte para 224 x 224
        img = kimage.load_img(fname, target_size=(px, px))
        # transforma imagem num array e carrega em X
        img_array = kimage.img_to_array(img)
        X[i - min_idx, :, :, :] = img_array

    # pre processa X (usa função keras.applications.vgg16.preprocess_input)
    X = preprocess_input(X)
    these_preds = model.predict(X)
    shp = ((max_idx - min_idx), n_dims) #shp = ((max_idx - min_idx) + 1, n_dims)
    
    dados["predicoes"][min_idx:max_idx, :] = these_preds.reshape(shp) #dados["predicoes"][min_idx:max_idx + 1, :] = these_preds.reshape(shp)
    
    min_idx = max_idx
    max_idx = np.min((max_idx + batch_size, total_max))


In [ ]:
import pickle
def grava_dados_pred():
    filehandler = open("dados_pred.pickle", 'wb') 
    pickle.dump(dados, filehandler)
grava_dados_pred()



In [ ]:
def le_dados_pred():
    filehandler = open("dados_pred.pickle", 'rb') 
    ret = pickle.load(filehandler)
    return ret

dados = le_dados_pred() 


In [ ]:
# lista devices de computacao disponiveis

from tensorflow.python.client import device_lib

def get_available_devices():  
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())  

In [ ]:
def comp_similarity(ratings):
    sim = ratings.dot(ratings.T)
    if not isinstance(sim, np.ndarray):
        sim = sim.toarray()
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [ ]:
import tensorflow as tf

def comp_similarity_tf2(ratings):
    sess = tf.Session()
    
    matrix1 = tf.placeholder(tf.float32, shape=ratings.shape)
    #matrix1 = tf.convert_to_tensor(ratings, np.float32)
    #matrix2 = tf.transpose(matrix1)
    sim = tf.matmul(matrix1, matrix1, transpose_b=True, a_is_sparse=True, b_is_sparse=True)
    diag0 = tf.diag_part(sim)
    diag = tf.zeros([1, diag0.get_shape()[0]], tf.float32)
    diag = tf.add(diag, diag0)
    norms = tf.sqrt(diag)
    normst = tf.transpose(norms)
    div1 = tf.divide(sim, norms)
    div2 = tf.divide(div1, normst)
    result = sess.run(div2, feed_dict={matrix1: ratings.toarray()})
    sess.close()
    return result


In [ ]:
#versão numpy
# transforma as predicoes em uma compressed sparse row (csr)
#dados["predicoescsr"] = dados["predicoes"].tocsr()
#dados["sim"] = cosine_similarity(dados["predicoescsr"])

#versão tensorflow
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
dados["sim"] = comp_similarity_tf2(dados["predicoes"])
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))


In [ ]:
dados["sim"][3][0:10]


In [ ]:

def get_similar_from_busca(dados, idx, N=5, minimoinicial=0.3, minimofinal=0.2):
    #exrr = np.array([  848,   1830,  1950,  1913,  2428,  3542, 3558, 3561, 3576, 3598, 3622,
    #                   3626, 3627, 3636, 3641, 3683, 3718,  3720,  3723, 3746, 3730, 
    #                   3770, 4391,  4560,  5201,  5264,  5478,  5664,  5881,  5918,  
    #                   6119,  6190,  6539,  6983,  7286])

    if idx>=len(dados["busca"]["itens"]):
        print("  Índice fornecido é de material:", idx)
        return [], [], []
    row = dados["sim"][idx, :]
    imgs = []
    mids = []
    indx = []
    valsim = []
    cont=0
    
    #mediadoc = np.average(row[exrr])
    #maxdoc = np.max(row[exrr])
    
    #if maxdoc>0.4:
    #    minimoinicial=0.85
    #    minimofinal=0.7

    if idx==8555:
        print("  idx=", idx)
        print("  matches ", dados["total"]["nomes"][idx])
        print("  ", np.argsort(-row)[0:30])
    for x in np.argsort(-row):
        if x>=len(dados["busca"]["itens"]):
            #if idx == 8555:
            #    print("x=", x, "row[x]=", row[x])
            if ((cont==0 and row[x]<minimoinicial) or row[x]<minimofinal): break
            #print(row[x])
            imgs.append(dados["total"]["nomes"][x])
            mids.append(dados["total"]["itens"][x][0])
            valsim.append(row[x])
            indx.append(x)
            cont += 1
            if cont>=N: break
    #if len(imgs)>0: print(idx, " - correlação documento:", mediadoc, maxdoc)
    return imgs, mids, valsim, indx


def get_similar_from_material(dados, idx, N=5, minimoinicial=0.3, minimofinal=0.2):
    #exrr = np.array([  848,   1830,  1950,  1913,  2428,  3542, 3558, 3561, 3576, 3598, 3622,
    #                   3626, 3627, 3636, 3641, 3683, 3718,  3720,  3723, 3746, 3730, 
    #                   3770, 4391,  4560,  5201,  5264,  5478,  5664,  5881,  5918,  
    #                   6119,  6190,  6539,  6983,  7286])

    if idx<len(dados["busca"]["itens"]):
        print("  Índice fornecido é de busca:", idx)
        return [], [], []
    row = dados["sim"][idx, :]
    imgs = []
    mids = []
    indx = []
    valsim = []
    cont=0
    
    #mediadoc = np.average(row[exrr])
    #maxdoc = np.max(row[exrr])
    
    #if maxdoc>0.4:
    #    minimoinicial=0.85
    #    minimofinal=0.7
    if idx==8555:
        print("  idx=", idx)
        print("  matches ", dados["total"]["nomes"][idx])
        print("  ", np.argsort(-row)[0:30])
    for x in np.argsort(-row):
        if x<len(dados["busca"]["itens"]):
            #if idx == 8555:
            #    print("x=", x, "row[x]=", row[x])
            if ((cont==0 and row[x]<minimoinicial) or row[x]<minimofinal): break
            #print(row[x])
            imgs.append(dados["total"]["nomes"][x])
            mids.append(dados["total"]["nomes"][x][0])
            valsim.append(row[x])
            indx.append(x)
            cont += 1
            if cont>=N: break
    #if len(imgs)>0: print(idx, " - correlação documento:", mediadoc, maxdoc)
    return imgs, mids, valsim, indx


In [ ]:


#relatório procura no material imagens similares às da busca
def gera_html_busca(dados, arqs, excl, N=5, minimoinicial=0.3, minimofinal=0.2):
    cont=0
    contsim=0
    nro_arq_busca = 0
    try:
        os.stat("relatorios")
    except:
        os.mkdir("relatorios")    

    try:
        os.stat("relatorios/imgreport_b")
    except:
        os.mkdir("relatorios/imgreport_b")    
        
    outF = open("relatorios/similares_a_busca.html", "w", encoding='utf-8')
    outF.write("<html>\n")
    outF.write("<head><meta charset=\"UTF-8\"></head>\n")
    outF.write("<style> \n\
    table {\n\
        border-collapse: collapse;\n\
        width: 100%;\n\
        background-color: #ffffff;\n\
    }\n\
    th, td {\n\
        border: 1px solid #ccc;\n\
        padding: 10px;\n\
    }\n\
    th {\n\
      background-color: #dadbda;\n\
    }\n\
    table.alt tr:nth-child(even) {\n\
        background-color: #eee;\n\
    }\n\
    table.alt tr:nth-child(odd) {\n\
        background-color: #fff;\n\
    }\n\
    .tabelamaior {\n\
       background-color: #fafbfa;\n\
     }\n\
    .titulo1 {\n\
	background-color: #dccccc;\n\
	height:60px;\n\
    }\n\
    .titulo2 {\n\
	background-color: #ccdccc;\n\
	height:60px;\n\
    }\n\
</style>\n")    
    
    outF.write("<body>\n")

    for ind in range (0, len(dados["busca"]["itens"])):
        if ind in excl: continue
        resimg, resmid, resvalsim, indx = get_similar_from_busca(dados, ind, N=N, minimoinicial=minimoinicial, minimofinal=minimofinal)
        if "20150519_095035.jpg" in dados["total"]["nomes"][ind]:
            print("  ", dados["total"]["nomes"][ind])
            print("  ", ind, resimg, resmid, resvalsim, indx)
        nro_arq_busca += 1
        if len(resimg)>0:
            #print(resimg, resmid, resvalsim, indx)
            try:
                img = mpimg.imread(dados["total"]["nomes"][ind])
                max = min(img.shape[0], img.shape[1])
                if max<40: 
                    #print("max<40", img.shape, dados["total"]["nomes"][ind])
                    continue
                outF.write("<table class=tabelamaior style=\"padding:10px\" valign=top><tr><th width=50%>Imagem do " + 
			arqs["busca"]["nome_exibicao"] + "</th><th width=50%>Imagens similares no " + 
			arqs["material"]["nome_exibicao"] + "</th></tr>\n")
                outF.write("<tr>\n\
      <td valign=top>\n\
      <table valign=top>\n")
                outF.write("    <tr><th class=titulo1 >" + dados["total"]["nomes"][ind][len(arqs["busca"]["raiz"]):] + "</th></tr>\n")
                outF.write("    <tr><td>Índice: " + str(ind) + "</td></tr>\n")
                outF.write("    <tr><td>Tamanho (bytes): " 
                           + str(os.path.getsize(dados["total"]["nomes"][ind])) + "</td></tr>\n")
                outF.write("    <tr><td>Dimensões da imagem (alturaxlargura): " + 
                             str(img.shape[0]) + "x" + str(img.shape[1]) + "</td></tr>\n")
                outF.write("    <tr><td>Hash (MD5): <font style=\"font-family:courier;\">" + 
                           dados["total"]["itens"][ind][0] + "</font></td></tr>\n")



                h = img.shape[0]
                w = img.shape[1]
                szmax=400
                if h>w and h>szmax: 
                    w = int(w * szmax / h)
                    h=szmax
                if w>h and w>szmax: 
                    h = int(h * szmax / w)
                    w=szmax

                novonome = "imgreport_b/" + dados["total"]["itens"][ind][0] + dados["total"]["nomes"][ind][dados["total"]["nomes"][ind].rfind("."):]
                try:
                    copyfile(dados["total"]["nomes"][ind], "relatorios/"+novonome)
                except Exception as ex:
                    print("Erro:", ex)
                    print("  erro no copyfile de", novonome)

                #outF.write("    <tr><td > <img src='" + dados["total"]["nomes"][ind].replace("%", "%25") + 
                outF.write("    <tr><td > <img src='" + novonome + 
                           "' width=" + str(w) + "px heigth=" + str(h) + "px></td></tr>\n")
                outF.write("  </table>\n  </td>\n  <td valign=top>\n")

                cont+=1

                for res in range(0, len(resimg)):
                    nomeimg = resimg[res]
                    #print(nomeimg)
                    valsim = resvalsim[res]
                    #if res==0 and valsim<nivel: limiar.append(ind)
                    try:
                        img = mpimg.imread(nomeimg)
                        contsim+=1

                        outF.write("  <table valign=top>\n")

                        outF.write("    <tr><th class=titulo2 >" + str(res+1) + "a imagem similar: " +
                                   nomeimg[len(arqs["material"]["raiz"]):] + "</th></tr>\n")

                        outF.write("    <tr><td>Índice: " + str(indx[res]) + " (Similaridade: " +
                                   "{:2.3f}".format(valsim) + ")</td></tr>\n")

                        outF.write("    <tr><td>Tamanho (bytes): " 
                                   + str(os.path.getsize(nomeimg)) + "</td></tr>\n")
                        outF.write("    <tr><td>Dimensões da imagem (alturaxlargura): " + 
                                     str(img.shape[0]) + "x" + str(img.shape[1]) + "</td></tr>\n")
                        outF.write("    <tr><td>Hash (MD5): <font style=\"font-family:courier;\">" + 
                                   dados["total"]["itens"][indx[res]][0] + "</font></td></tr>\n")


                        h = img.shape[0]
                        w = img.shape[1]
                        szmax=400
                        if h>w and h>szmax: 
                            w = int(w * szmax / h)
                            h=szmax
                        if w>h and w>szmax: 
                            h = int(h * szmax / w)
                            w=szmax
                        novonome = "imgreport_b/" + dados["total"]["itens"][indx[res]][0] + nomeimg[nomeimg.rfind("."):]
                        try:
                            copyfile(nomeimg, "relatorios/"+novonome)
                        except Exception as ex:
                            print("erro:", ex)
                            print("  erro no copyfile de", novonome)

                        #outF.write("    <tr><td > <img src='" + nomeimg.replace("%", "%25") + 
                        outF.write("    <tr><td > <img src='" + novonome + 
                                   "' width=" + str(w) + "px heigth=" + str(h) + "px></td></tr>\n")
                        outF.write("  </table>\n  <br>\n")
                    except Exception as ex2:
                        print("\n  Erro ", ex2)
                        print("   -- imreag ", nomeimg)


                outF.write("  </td>\n</tr>\n</table>\n<br>\n\n")                

                outF.write("<hr style=\"border=2\">")

                #if cont>30: break
            
            except Exception as eximread:
                print("  Erro ", eximread) 
                print("   -- imreag ", dados["total"]["nomes"][ind])
    outF.write("</body>\n")
    outF.write("</html>\n")
    outF.close()
    print("  ", cont, contsim, nro_arq_busca)    
    


#relatório procura na busca imagens similares às do material 
def gera_html_material(dados, arqs, excl, N=5, minimoinicial=0.3, minimofinal=0.2):
    cont=0
    contsim=0

    try:
        os.stat("relatorios")
    except:
        os.mkdir("relatorios")    

    try:
        os.stat("relatorios/imgreport_m")
    except:
        os.mkdir("relatorios/imgreport_m")    
        
    outF = open("relatorios/similares_ao_material.html", "w", encoding='utf-8')
    outF.write("<html>\n")
    outF.write("<head><meta charset=\"UTF-8\"></head>\n")
    outF.write("<style> \n\
    table {\n\
        border-collapse: collapse;\n\
        width: 100%;\n\
        background-color: #ffffff;\n\
    }\n\
    th, td {\n\
        border: 1px solid #ccc;\n\
        padding: 10px;\n\
    }\n\
    th {\n\
      background-color: #dadbda;\n\
    }\n\
    table.alt tr:nth-child(even) {\n\
        background-color: #eee;\n\
    }\n\
    table.alt tr:nth-child(odd) {\n\
        background-color: #fff;\n\
    }\n\
    .tabelamaior {\n\
       background-color: #fafbfa;\n\
     }\n\
    .titulo1 {\n\
	background-color: #dccccc;\n\
	height:60px;\n\
    }\n\
    .titulo2 {\n\
	background-color: #ccdccc;\n\
	height:60px;\n\
    }\n\
</style>\n")    
    
    outF.write("<body>\n")

    for ind in range (len(dados["busca"]["itens"]), len(dados["total"]["itens"])):
        if ind in excl: continue
        resimg, resmid, resvalsim, indx = get_similar_from_material(dados, ind, N=N, minimoinicial=minimoinicial, minimofinal=minimofinal)
        #if "20150519_095035.jpg" in dados["total"]["nomes"][ind]:
        #    print(dados["total"]["nomes"][ind])
        #    print(ind, resimg, resmid, resvalsim, indx)
        if len(resimg)>0:
            #print("  ", dados["total"]["nomes"][ind])
            try:
                img = mpimg.imread(dados["total"]["nomes"][ind])
                max = min(img.shape[0], img.shape[1])
                if max<40: 
                    #print("max<40", img.shape, dados["total"]["nomes"][ind])
                    continue
                outF.write("<table class=tabelamaior style=\"padding:10px\" valign=top><tr><th width=50%>Imagem do " + 
			arqs["material"]["nome_exibicao"] + "</th><th width=50%>Imagens similares no " +
			arqs["busca"]["nome_exibicao"] + "</th></tr>\n")
                outF.write("<tr>\n\
      <td valign=top>\n\
      <table valign=top>\n")
                outF.write("    <tr><th class=titulo1 >" + dados["total"]["nomes"][ind][len(arqs["material"]["raiz"]):] + "</th></tr>\n")
                outF.write("    <tr><td>Índice: " + str(ind) + "</td></tr>\n")
                outF.write("    <tr><td>Tamanho (bytes): " 
                           + str(os.path.getsize(dados["total"]["nomes"][ind])) + "</td></tr>\n")
                outF.write("    <tr><td>Dimensões da imagem (alturaxlargura): " + 
                             str(img.shape[0]) + "x" + str(img.shape[1]) + "</td></tr>\n")
                outF.write("    <tr><td>Hash (MD5): <font style=\"font-family:courier;\">" + 
                           dados["total"]["itens"][ind][0] + "</font></td></tr>\n")



                h = img.shape[0]
                w = img.shape[1]
                szmax=400
                if h>w and h>szmax: 
                    w = int(w * szmax / h)
                    h=szmax
                if w>h and w>szmax: 
                    h = int(h * szmax / w)
                    w=szmax

                novonome = "imgreport_m/" + dados["total"]["itens"][ind][0] + dados["total"]["nomes"][ind][dados["total"]["nomes"][ind].rfind("."):]
                try:
                    copyfile(dados["total"]["nomes"][ind], "relatorios/"+novonome)
                except Exception as ex:
                    print("Erro:", ex)
                    print("  erro no copyfile de", novonome)

                #outF.write("    <tr><td > <img src='" + dados["total"]["nomes"][ind].replace("%", "%25") + 
                outF.write("    <tr><td > <img src='" + novonome + 
                           "' width=" + str(w) + "px heigth=" + str(h) + "px></td></tr>\n")
                outF.write("  </table>\n  </td>\n  <td valign=top>\n")

                cont+=1



                for res in range(0, len(resimg)):
                    nomeimg = resimg[res]
                    #print(nomeimg)
                    valsim = resvalsim[res]
                    #if res==0 and valsim<nivel: limiar.append(ind)
                    try:
                        img = mpimg.imread(nomeimg)
                        contsim+=1

                        outF.write("  <table valign=top>\n")

                        outF.write("    <tr><th class=titulo2 >" + str(res+1) + "a imagem similar: " +
                                nomeimg[len(arqs["busca"]["raiz"]):] + "</th></tr>\n")

                        outF.write("    <tr><td>Índice: " + str(indx[res]) + " (Similaridade: " +
                                "{:2.3f}".format(valsim) + ")</td></tr>\n")

                        outF.write("    <tr><td>Tamanho (bytes): " 
                                + str(os.path.getsize(nomeimg)) + "</td></tr>\n")
                        outF.write("    <tr><td>Dimensões da imagem (alturaxlargura): " + 
                                    str(img.shape[0]) + "x" + str(img.shape[1]) + "</td></tr>\n")
                        outF.write("    <tr><td>Hash (MD5): <font style=\"font-family:courier;\">" + 
                                dados["total"]["itens"][indx[res]][0] + "</font></td></tr>\n")


                        h = img.shape[0]
                        w = img.shape[1]
                        szmax=400
                        if h>w and h>szmax: 
                            w = int(w * szmax / h)
                            h=szmax
                        if w>h and w>szmax: 
                            h = int(h * szmax / w)
                            w=szmax
                        novonome = "imgreport_m/" + dados["total"]["itens"][indx[res]][0] + nomeimg[nomeimg.rfind("."):]
                        try:
                            copyfile(nomeimg, "relatorios/"+novonome)
                        except:
                            print("  erro no copyfile de", novonome)

                        #outF.write("    <tr><td > <img src='" + nomeimg.replace("%", "%25") + 
                        outF.write("    <tr><td > <img src='" + novonome + 
                                "' width=" + str(w) + "px heigth=" + str(h) + "px></td></tr>\n")
                        outF.write("  </table>\n  <br>\n")
                    except Exception as ex2:
                        print("\n  Erro ", ex2)
                        print("   -- imreag ", nomeimg)


                outF.write("  </td>\n</tr>\n</table>\n<br>\n\n")                

                outF.write("<hr style=\"border=2\">")

                #if cont>30: break
            
            except Exception as eximread:
                print("  Erro ", eximread) 
                print("   -- imreag ", dados["total"]["nomes"][ind])
    outF.write("</body>\n")
    outF.write("</html>\n")
    outF.close()
    print("  ", cont, contsim)    




In [ ]:
print("relatorio busca")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

excl = [ ]
gera_html_busca(dados, arqs, excl, N=3, minimoinicial=0.20, minimofinal=0.12)

print("Fim")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))


In [ ]:
print("relatorio material")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

excl = [ ]
gera_html_material(dados, arqs, excl, N=3, minimoinicial=0.2, minimofinal=0.12)

print("Fim")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
